**Progetto Machine Learning**
---
**Giovanni D'Agostino, Marco Giorgi**

Fase Preliminare: 
- individuare la GPU Tesla-P100
- montare il drive contenente il codice e i dataset

In [ ]:
########## Tipo GPU allocata #############
!nvidia-smi

Tue Dec  1 09:50:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
import sys
drive.mount('Project_ML')
sys.path.append('/content/Project_ML/My Drive/ML/ProveFinali/Dishwasher')

Mounted at Project_ML


# **Load Dataset**



Fase di Caricamento:
- import librerie
- load dei dataset main e dishwasher
- descrizione base dei dataset


In [ ]:
import preprocessing 
import pandas as pd
import model
import metrics
import generator
import stats_data
import tensorflow as tf
import numpy as np

In [ ]:
pd.set_option('float_format', '{:f}'.format)
df_main = pd.read_csv("/content/Project_ML/My Drive/ML/ProveFinali/main_train.csv")
df_main.describe(include='all')

,timestamp,power
count,6082508,6082508.000000
unique,6082508,nan
top,2019-02-14 21:23:28+01:00,nan
freq,1,nan
mean,NaN,370.915554
std,NaN,549.188099
min,NaN,73.481000
25%,NaN,167.012000
50%,NaN,236.648000
75%,NaN,356.898000


In [ ]:
df_dishwasher = pd.read_csv("/content/Project_ML/My Drive/ML/ProveFinali/dishwasher_train.csv")
df_dishwasher.describe(include='all')

,timestamp,power
count,6082508,6082508.000000
unique,6082508,nan
top,2019-02-14 21:23:28+01:00,nan
freq,1,nan
mean,NaN,25.871098
std,NaN,238.580181
min,NaN,0.000000
25%,NaN,0.000000
50%,NaN,0.000000
75%,NaN,0.000000


# **Sperimentazione**

Fase di sperimentazione:
- configurazione size di window e batch
- preprocessing (padding + normalizzazione)
- statistiche basi 
- modello
- configurazione generatore
- configurazione early stopping
- fit
- plot risultati

In [ ]:
window_size= 999 # NB!deve avere dimensione dispari
batch_size = 512

In [ ]:
main_power = preprocessing.load_data(df_main)
train_main, val_main = preprocessing.split_data(main_power)
min_main=np.min(train_main) #### min main train
max_main= np.max(train_main)  ##### max main train
train_main_pad = preprocessing.padding(train_main, window_size)
val_main_pad = preprocessing.padding(val_main, window_size)
train_main_norm = preprocessing.normalizzazione(train_main_pad, min_main, max_main)
val_main_norm = preprocessing.normalizzazione(val_main_pad, min_main, max_main)

dishwasher_power= preprocessing.load_data(df_dishwasher)
train_dish, val_dish = preprocessing.split_data(dishwasher_power)
min_dish=np.min(train_dish) ##### min dishwasher train
max_dish=np.max(train_dish) ##### max dishwasher train
train_dish_pad= preprocessing.padding(train_dish, window_size)
val_dish_pad= preprocessing.padding(val_dish, window_size)
train_dishwasher_norm = preprocessing.normalizzazione(train_dish_pad, min_dish, max_dish)
val_dishwasher_norm = preprocessing.normalizzazione(val_dish_pad, min_dish, max_dish)


In [ ]:
preprocessing.statDish("Main_Train", "dishwasher_Train",train_main, train_dish)
preprocessing.statDish("Main_Val", "dishwasher_Val",val_main, val_dish)

+-----+-------------------+------------------+
|     |     Main_Train    | dishwasher_Train |
+-----+-------------------+------------------+
| min | 73.48100000000002 |       0.0        |
| max | 6048.699999999999 |      2545.3      |
+-----+-------------------+------------------+
+-----+----------+----------------+
|     | Main_Val | dishwasher_Val |
+-----+----------+----------------+
| min |  92.725  |      0.0       |
| max |  5719.1  |     2570.6     |
+-----+----------+----------------+


In [ ]:
model_dishwasher = model.dishwasher_model(window_size, metrics.F1_score(scaling='norm', Min=min_dish, Max=max_dish))
model_dishwasher.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 990, 30)           330       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 983, 40)           9640      
_________________________________________________________________
flatten (Flatten)            (None, 39320)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               20132352  
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 20,142,835
Trainable params: 20,142,835
Non-trainable params: 0
_________________________________________________________________


In [ ]:
train_generator = generator.DataGenerator(train_main_norm, train_dishwasher_norm, window_len=window_size, batch_len=batch_size)
val_generator = generator.DataGenerator(val_main_norm, val_dishwasher_norm, window_len=window_size, batch_len=batch_size, shuffle=False)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_f1_score', mode='max', patience=5)
callbacks = [early_stopping]

In [ ]:
fit_data = model_dishwasher.fit(x=train_generator, epochs=30, steps_per_epoch= train_generator.__len__(), validation_data = val_generator, validation_steps= val_generator.__len__(), callbacks= callbacks )
stats_data.plot_fit(fit_data)